# UnSupervisedIOHMM

In [1]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

## Load data

In [2]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()

input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

data.head()
print(len(data))


input = torch.tensor(np.array(data[['Open', 'High', 'Low', 'Close']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close-next-day']), dtype=torch.float32)

print(output[0:5])
print(input[0:5])
torch.cat((torch.tensor([1.0]), output))



3259
tensor([1.5887, 1.4640, 1.2800, 1.0740, 1.0533])
tensor([[1.2667, 1.6667, 1.1693, 1.5927],
        [1.7193, 2.0280, 1.5533, 1.5887],
        [1.6667, 1.7280, 1.3513, 1.4640],
        [1.5333, 1.5400, 1.2473, 1.2800],
        [1.3333, 1.3333, 1.0553, 1.0740]])


tensor([  1.0000,   1.5887,   1.4640,  ..., 224.5700, 234.8600, 244.4000])

# Example 1

## Set up a simple model manully

In [3]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=1000, tol=1e-6)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[ 0.5371,  0.7206, -0.9755, -0.4663, -1.6678],
         [ 1.2311,  1.8702, -1.2788,  1.3563,  1.0594]],

        [[-1.1267, -0.7439,  0.2039,  0.1396, -0.6601],
         [-0.2718, -0.0351,  0.6029,  2.3986,  0.2993]]], requires_grad=True)
Parameter containing:
tensor([[ 0.1097,  0.5070, -0.7417, -0.3175,  0.3450],
        [-0.6920,  0.8701, -1.1825,  0.9215,  0.3023]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [4]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[3.7040e-01, 6.2960e-01],
        [2.4994e-04, 9.9975e-01],
        [1.0559e-04, 9.9989e-01],
        [2.7061e-04, 9.9973e-01],
        [6.1553e-04, 9.9938e-01],
        [1.5879e-03, 9.9841e-01],
        [2.3150e-03, 9.9768e-01],
        [1.6729e-03, 9.9833e-01],
        [1.2480e-03, 9.9875e-01],
        [1.1623e-03, 9.9884e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [5]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[2.4994e-04, 9.9975e-01],
        [1.0559e-04, 9.9989e-01],
        [2.7061e-04, 9.9973e-01],
        [6.1553e-04, 9.9938e-01],
        [1.5879e-03, 9.9841e-01],
        [2.3150e-03, 9.9768e-01],
        [1.6729e-03, 9.9833e-01],
        [1.2480e-03, 9.9875e-01],
        [1.1623e-03, 9.9884e-01],
        [3.8948e-01, 6.1052e-01]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [6]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[1.4706e-04, 9.9985e-01],
        [2.6401e-08, 1.0000e+00],
        [2.8586e-08, 1.0000e+00],
        [1.6672e-07, 1.0000e+00],
        [9.7958e-07, 1.0000e+00],
        [3.6905e-06, 1.0000e+00],
        [3.8884e-06, 1.0000e+00],
        [2.0940e-06, 1.0000e+00],
        [1.4540e-06, 1.0000e+00],
        [7.4176e-04, 9.9926e-01]])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [7]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[1.2182e-10, 5.0000e-01],
         [1.2182e-10, 5.0000e-01]],

        [[7.2070e-09, 5.0000e-01],
         [7.2070e-09, 5.0000e-01]],

        [[1.6148e-11, 5.0000e-01],
         [1.6148e-11, 5.0000e-01]],

        [[2.9048e-11, 5.0000e-01],
         [2.9048e-11, 5.0000e-01]],

        [[4.7948e-10, 5.0000e-01],
         [4.7948e-10, 5.0000e-01]],

        [[2.3653e-09, 5.0000e-01],
         [2.3653e-09, 5.0000e-01]],

        [[3.4688e-09, 5.0000e-01],
         [3.4688e-09, 5.0000e-01]],

        [[3.0316e-09, 5.0000e-01],
         [3.0316e-09, 5.0000e-01]],

        [[1.9313e-09, 5.0000e-01],
         [1.9313e-09, 5.0000e-01]],

        [[7.3932e-07, 5.0000e-01],
         [7.3932e-07, 5.0000e-01]]])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [8]:
#IOHMM._baum_welch()

## See the training results

In [9]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[ 0.5371,  0.7206, -0.9755, -0.4663, -1.6678],
         [ 1.2311,  1.8702, -1.2788,  1.3563,  1.0594]],

        [[-1.1267, -0.7439,  0.2039,  0.1396, -0.6601],
         [-0.2718, -0.0351,  0.6029,  2.3986,  0.2993]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[ 0.1097,  0.5070, -0.7417, -0.3175,  0.3450],
        [-0.6920,  0.8701, -1.1825,  0.9215,  0.3023]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)


## Viterbi

In [10]:
#call the viterbi algorithm
IOHMM.viterbi()

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

# Predictions

In [11]:
emission_pred = IOHMM.predict(input[2])
print(emission_pred)


torch.Size([2, 5])
torch.Size([4])
(2,)


RuntimeError: The size of tensor a (5) must match the size of tensor b (4) at non-singleton dimension 1